In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import xarray as xr
import pandas as pd
from pyproj import Geod
import pygmt
import obspy

# Coordinates

In [ ]:
def grdmath(args:str, **kwargs):
    # Create a session and destroy it automatically when exiting the "with" block.
    with pygmt.clib.Session() as ses:
        # Create a temp file to use as output.
        with pygmt.helpers.GMTTempFile(suffix='.nc') as temp:
            # Call the grdinfo module with the temp file as output.
            ses.call_module(
                "grdmath", f"{args} = {temp.name}"
            )
            grd = xr.open_dataset(temp.name, **kwargs)
            grd.load()
    return grd

In [ ]:
def columns_at_end(df : pd.DataFrame, cols_at_end: list):
    return df[[c for c in df if c not in cols_at_end] + [c for c in cols_at_end if c in df]]


def columns_at_begin(df : pd.DataFrame, cols_at_begin: list):
    return df[[c for c in cols_at_begin if c in df] + [c for c in df if c not in cols_at_begin]]


def mergeR2A(df : pd.DataFrame, array : str, glob: bool, used: bool):
    i = df.index[df.station.str.startswith(array, na=False)]
    df.loc[i, 'used'] = used
    e = df.loc[i]
    a = e.iloc[0].to_dict()
    a['station'] = array
    a['longitude'] = e.longitude.mean()
    a['latitude'] = e.latitude.mean()
    a['elevation'] = e.elevation.mean()
    a['local_depth'] = e.local_depth.mean()
    a['type'] = 'A'
    a['used'] = used
    a['glob'] = glob
    return df.append(a, ignore_index=True)

In [ ]:
inv = obspy.read_inventory('../datasets/station_inventory_map.xml')
t = '2015-01-01'

In [ ]:
df = []
for seed_id in sorted(set(inv.get_contents()['channels'])):
    tmp = inv.get_coordinates(seed_id,t)
    tmp['network'], tmp['station'], tmp['location'], tmp['channel'] = seed_id.split('.')
    tmp['type'] = 'R'
    tmp['glob'] = tmp['channel'][:2] == 'BH'
    tmp['used'] = True
    df.append(tmp)
# add source    
df.append(
    {'station': 'MVC', 'latitude': -25.887, 'longitude': -177.188, 'elevation': 0., 'local_depth': 132., 
     'type': 'S', 'channel': '', 'location': '', 'network': '', 'glob': True, 'used': True}
)

df = pd.DataFrame(df)

df.loc[(df.station == 'RAO') | (df.station == 'AFI') | (df.station == 'NFK') | (df.station == 'NIUE'), 'used'] = False

# merge Hydro sensors to array
df = mergeR2A(df, 'H03S', glob=True, used=True)
df = mergeR2A(df, 'H03N', glob=False, used=False)
df = mergeR2A(df, 'H10N', glob=True, used=True)
df = mergeR2A(df, 'H10S', glob=False, used=False)

df['distance'] = 0.
df['bearing'] = 0.

df.loc[(df.elevation == 0.), ['elevation']] = - df.local_depth[(df.elevation == 0.)]

df = columns_at_begin(df,['longitude', 'latitude', 'elevation', 'local_depth'])

g = Geod(ellps='WGS84')
mvc = df.loc[df.type=='S'].iloc[0]

for i in df.index:
    az12,az21,dis = g.inv( 
        df.iloc[i]['longitude'], df.iloc[i]['latitude'],
        mvc['longitude'], mvc['latitude']
    )
    df.loc[i,'bearing'] = az12 + 360. if az12 < 0. else az12
    df.loc[i,'distance'] = dis/1e3

# Map

In [ ]:
def plot_station_name(fig, coord:pd.DataFrame, **kwargs):
    fig.text(x=coord.longitude, y=coord.latitude, text=coord.station, **kwargs)
    return

In [ ]:
grid = pygmt.datasets.load_earth_relief(resolution="15s", region=[-85, -75, -35, -30],)

In [ ]:
s = df.loc[df.type=='S']
g = df.loc[df.glob]
h = df.loc[df.station.str.contains(r"H\d{2}\w\d")]

config = dict(FONT_ANNOT_PRIMARY="8p", FONT_ANNOT_SECONDARY="8p", FONT_LABEL="9p", FONT_TITLE="9p")

rotate = '81'
font = '6.5p,Helvetica-Bold,black+a-81'

fig = pygmt.Figure()

# globe
fig.coast(region='g', projection='S-90/-65/9c', frame='a', land='gray', p=rotate)

sdist = grdmath(f"-Rg -I1 {s.longitude.values[0]} {s.latitude.values[0]} SDIST")
sdist.to_netcdf('sdist.nc', 'w')  # quick fix for failing in-memory solution

fig.grdcontour(grid="sdist.nc", annotation='2000+ukm+f6',
               W=['athinner,black', 'cthinner,black,-'], C=1000, G='lz-/z+', p=rotate)

fig.plot(x=s.longitude, y=s.latitude, style='E-3690k', pen='thin,firebrick2,-', p=rotate)

for i, r in g.loc[df.type != 'S'].iterrows():
    pair = g.loc[ (g.type == 'S') | ( g.station == r.station)]
    fig.plot(x=pair.longitude, y=pair.latitude,
             pen='thin,steelblue2' if r.channel == 'EDH' else 'thin,firebrick2', p=rotate)

fig.plot(x=g.loc[g.channel == 'EDH'].longitude, y=g.loc[g.channel == 'EDH'].latitude, style="c0.15c", color="steelblue3", pen="thick,steelblue3", p=rotate)
fig.plot(x=g.loc[(g.channel == 'BHZ') & (g.used == True)].longitude, y=g.loc[(g.channel == 'BHZ') & (g.used == True)].latitude, style="c0.15c", color="firebrick3", pen="thick,firebrick3", p=rotate)
fig.plot(x=g.loc[(g.channel == 'BHZ') & (g.used == False)].longitude, y=g.loc[(g.channel == 'BHZ') & (g.used == False)].latitude, style="c0.15c", color="white", pen="thick,firebrick3", p=rotate)
fig.plot(x=s.longitude, y=s.latitude, style="a0.35c", color="orange", pen="thinner,black", p=rotate)

plot_station_name(fig, g.loc[g.station=='MVC'], G='white', D='-.3c/-.1c', justify='TC', font=font, p=rotate)

plot_station_name(fig, g.loc[g.station=='RAR'], G='white', D='.05c/.3cv', justify='MR', font=font, p=rotate)
plot_station_name(fig, g.loc[g.station=='AFI'], G='white', D='-.35c/.4cv', justify='MR', font=font, p=rotate)
plot_station_name(fig, g.loc[g.station=='NFK'], G='white', D='0.05c/-.5cv', justify='ML', font=font, p=rotate)
plot_station_name(fig, g.loc[g.station=='NIUE'], G='white', D='-.3c/.7cv', justify='MR', font=font, p=rotate)
plot_station_name(fig, g.loc[g.station=='RAO'], G='white', D='.5c/-.5cv', justify='ML', font=font, p=rotate)

plot_station_name(fig, g.loc[g.station=='H03S'], G='white', D='-.2c/.2c', justify='MR', font=font, p=rotate)
plot_station_name(fig, g.loc[g.station=='H10N'], G='white', D='-.2c/.2c', justify='MR', font=font, p=rotate)


# H03 zoom
fig.grdcontour(region='-79.7/-78.6/-34.05/-33.35', projection='q-78.9/6c',
               grid=grid.clip(max=-50), annotation='1000+f6',
               W=['athinner,gray60', 'cthinnest,gray80,-'], C=250,
               xshift="1.8c", yshift="5c")
fig.coast(land='black')

pair = df.loc[ (df.type == 'S') | ( df.station == "H03S")]
fig.plot(x=pair.longitude, y=pair.latitude, pen='thinner,steelblue2')
pair = df.loc[ (df.type == 'S') | ( df.station == "H03N")]
fig.plot(x=pair.longitude, y=pair.latitude, pen='thinner,steelblue2,-')

fig.plot(x=h[h.used].longitude, y=h[h.used].latitude, style="c2.5p", color="steelblue3", pen="thin,steelblue2")
fig.plot(x=h[~h.used].longitude, y=h[~h.used].latitude, style="c2.5p",  color="white", pen="thin,steelblue2")

plot_station_name(fig, df.loc[df.station=='H03S'], G='white', D='-.1c/.3c', justify='MR', font="7.5p,Helvetica-Bold,black")
plot_station_name(fig, df.loc[df.station=='H03N'], G='white', D='-.1c/.3c', justify='MR', font="7.5p,Helvetica-Bold,black")
fig.text(y=-33.6361, x=-78.89185, text="Juan Fernández Islands", G='white', D='0c/.5c', justify='CM', font="6.5p,Helvetica,black")

with pygmt.config(MAP_FRAME_TYPE="plain", **config):
    fig.basemap(frame=['a15mf05m', 'NEsw'], map_scale='JTL+o.5c/-.5c+c-33.6+w10+jbl+l')

fig.text(text="b)", D='-.3c/.3c', position='TL', justify='MR', font="10p,Helvetica-Bold,black", no_clip=True)
fig.text(text="a)", D='-9.7c/.3c', position='TL', justify='MR', font="10p,Helvetica-Bold,black", no_clip=True)


# H03S layout
with pygmt.config(MAP_FRAME_TYPE="plain", **config):
    fig.basemap(region='-1750/1750/-1750/1750', projection='X3c', 
                frame=['a1000f500', 'nESw', "x+l\"x [m]\"", "y+l\"y [m]\""],
                xshift="0c", yshift="-3.5c", )

h03s = pd.DataFrame({
    'x': [1000.6, -1.2, -999.4],
    'y': [576.1, -1154.4, 578.3],
    'name': ['H03S1', 'H03S2', 'H03S3'],
    'offset': ['+4.3m', '-11.7m', '+7.3m']
})

fig.plot(x=h03s.x, y=h03s.y, style="c0.2c", color="steelblue3", pen="thick,steelblue2")
fig.text(x=h03s.x, y=h03s.y, text=h03s.name, D='0c/.25c', justify='CB', font="7.5p,Helvetica,black")
fig.text(x=h03s.x, y=h03s.y, text=h03s.offset, D='0c/-.25c', justify='CT', font="6p,Helvetica,black")
fig.text(text="mean depth 827.6m", D='.1c/-.1c', position='TL', justify='TL', font="6p,Helvetica,black")

fig.text(text="c)", D='-.3c/.3c', position='TL', justify='MR', font="10p,Helvetica-Bold,black", no_clip=True)


# fig.show()
fig.savefig("figs/map.pdf", show=True)

# cleanup
os.remove('sdist.nc')